In [1]:
introspection = ['type','getattr','hasattr', 'isinstance', 'issubclass',
                 'vars', 'callable', 'locals', 'globals', 'super']
object_changes = ['setattr', 'delattr','del']
code_generation = ['compile', 'eval''exec', 'execfile']
library_loading = ['import','reload']

In [7]:
# extract data with release filter
import os
import csv
# import pandas as pd
from pydriller import RepositoryMining
from radon.raw import analyze
from radon.metrics import h_visit
from radon.metrics import h_visit_ast
from radon.complexity import sorted_results
from pydriller.git_repository import GitRepository

fields = ['CommitID','filename','complexity','token_count','loc','lloc','sloc','comments',
          'multi','blank','code_comment','h1','h2','N1','N2','vocabulary','length',
          'calculated_length','volume', 'difficulty','effort','time','bugs',
          'change_type','added','removed','introspection','object_changes',
          'code_generation','library_loading']
introspection = ['type','getattr','hasattr', 'isinstance', 'issubclass',
                 'vars', 'callable', 'locals', 'globals', 'super']
object_changes = ['setattr', 'delattr','del']
code_generation = ['compile', 'eval''exec', 'execfile']
library_loading = ['import','reload']

repo_updated = "/Users/sanika/Downloads/Master Thesis/repo_updated/pytest"

count = 0
introspection_count = 0
object_changes_count = 0
code_generation_count = 0
library_loading_count = 0
# buggy_list = []

with open('final_dataset_03.csv', 'w') as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames = fields)
    writer.writeheader()
    for commit in RepositoryMining(repo_updated,
                                   only_modifications_with_file_types=['.py'] ).traverse_commits():
        for modification in commit.modifications:
            filename = modification.filename
            hash_val = commit.hash
            token_count = modification.token_count 
            complexity = modification.complexity
            if filename.endswith(".py"):
                #Calculate metrics for all commits
                for r, d, f in os.walk(repo_updated):
                    for file in f:
                        if filename in file:
                            file_path = os.path.join(r, file)
                            with open(file_path) as f:
                                content = f.read()
                                file_analyze = analyze(content)
                                code_n_comment = file_analyze.loc+file_analyze.comments
                                file_ast = h_visit(content)
                                for item1 in introspection:
                                    if item in content:
                                        introspection_count += 1
                                for item2 in object_changes:
                                    if item2 in content:
                                        object_changes_count += 1                                        
                                for item3 in code_generation:
                                    if item3 in content:
                                        code_generation_count += 1
                                for item4 in library_loading:
                                    if item4 in content:
                                        library_loading_count += 1
                                data = [{'CommitID':(hash_val), 
                                         'filename':(file_path),
                                         'complexity':(complexity),
                                     'token_count':(token_count),
                                     'loc':(file_analyze.loc),
                                     'lloc':(file_analyze.lloc),
                                     'sloc':(file_analyze.sloc),
                                     'comments':(file_analyze.comments),
                                     'multi':(file_analyze.multi),
                                     'blank':(file_analyze.blank),
                                     'code_comment':(code_n_comment),
                                     'h1':(file_ast.total.h1),
                                     'h2':(file_ast.total.h2),
                                     'N1':(file_ast.total.N1),
                                     'N2':(file_ast.total.N2),
                                     'vocabulary':(file_ast.total.vocabulary),
                                     'length':(file_ast.total.length),
                                     'calculated_length':(file_ast.total.calculated_length),
                                     'volume':(file_ast.total.volume),                     
                                     'difficulty':(file_ast.total.difficulty),
                                     'effort':(file_ast.total.effort),
                                     'time':(file_ast.total.time),
                                     'bugs':(file_ast.total.bugs),
                                     'change_type':(modification.change_type),
                                     'added':(modification.added),
                                     'removed':(modification.removed),                                               
                                     'introspection':(introspection_count),
                                     'object_changes':(object_changes_count),
                                     'code_generation':(code_generation_count),
                                     'library_loading':(library_loading_count)}]
                                count += 1
                                print(count)
                                writer.writerows(data)  
print("writing completed")
csvFile.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073


3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729


5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380


6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026


...

69156
69157
69158
69159
69160
69161
69162
69163
69164
69165
69166
69167
69168
69169
69170
69171
69172
69173
69174
69175
69176
69177
69178
69179
69180
69181
69182
69183
69184
69185
69186
69187
69188
69189
69190
69191
69192
69193
69194
69195
69196
69197
69198
69199
69200
69201
69202
69203
69204
69205
69206
69207
69208
69209
69210
69211
69212
69213
69214
69215
69216
69217
69218
69219
69220
69221
69222
69223
69224
69225
69226
69227
69228
69229
69230
69231
69232
69233
69234
69235
69236
69237
69238
69239
69240
69241
69242
69243
69244
69245
69246
69247
69248
69249
69250
69251
69252
69253
69254
69255
69256
69257
69258
69259
69260
69261
69262
69263
69264
69265
69266
69267
69268
69269
69270
69271
69272
69273
69274
69275
69276
69277
69278
69279
69280
69281
69282
69283
69284
69285
69286
69287
69288
69289
69290
69291
69292
69293
69294
69295
69296
69297
69298
69299
69300
69301
69302
69303
69304
69305
69306
69307
69308
69309
69310
69311
69312
69313
69314
69315
69316
69317
69318
69319
69320
69321
6932

70522
70523
70524
70525
70526
70527
70528
70529
70530
70531
70532
70533
70534
70535
70536
70537
70538
70539
70540
70541
70542
70543
70544
70545
70546
70547
70548
70549
70550
70551
70552
70553
70554
70555
70556
70557
70558
70559
70560
70561
70562
70563
70564
70565
70566
70567
70568
70569
70570
70571
70572
70573
70574
70575
70576
70577
70578
70579
70580
70581
70582
70583
70584
70585
70586
70587
70588
70589
70590
70591
70592
70593
70594
70595
70596
70597
70598
70599
70600
70601
70602
70603
70604
70605
70606
70607
70608
70609
70610
70611
70612
70613
70614
70615
70616
70617
70618
70619
70620
70621
70622
70623
70624
70625
70626
70627
70628
70629
70630
70631
70632
70633
70634
70635
70636
70637
70638
70639
70640
70641
70642
70643
70644
70645
70646
70647
70648
70649
70650
70651
70652
70653
70654
70655
70656
70657
70658
70659
70660
70661
70662
70663
70664
70665
70666
70667
70668
70669
70670
70671
70672
70673
70674
70675
70676
70677
70678
70679
70680
70681
70682
70683
70684
70685
70686
70687
7068

71892
71893
71894
71895
71896
71897
71898
71899
71900
71901
71902
71903
71904
71905
71906
71907
71908
71909
71910
71911
71912
71913
71914
71915
71916
71917
71918
71919
71920
71921
71922
71923
71924
71925
71926
71927
71928
71929
71930
71931
71932
71933
71934
71935
71936
71937
71938
71939
71940
71941
71942
71943
71944
71945
71946
71947
71948
71949
71950
71951
71952
71953
71954
71955
71956
71957
71958
71959
71960
71961
71962
71963
71964
71965
71966
71967
71968
71969
71970
71971
71972
71973
71974
71975
71976
71977
71978
71979
71980
71981
71982
71983
71984
71985
71986
71987
71988
71989
71990
71991
71992
71993
71994
71995
71996
71997
71998
71999
72000
72001
72002
72003
72004
72005
72006
72007
72008
72009
72010
72011
72012
72013
72014
72015
72016
72017
72018
72019
72020
72021
72022
72023
72024
72025
72026
72027
72028
72029
72030
72031
72032
72033
72034
72035
72036
72037
72038
72039
72040
72041
72042
72043
72044
72045
72046
72047
72048
72049
72050
72051
72052
72053
72054
72055
72056
72057
7205

73259
73260
73261
73262
73263
73264
73265
73266
73267
73268
73269
73270
73271
73272
73273
73274
73275
73276
73277
73278
73279
73280
73281
73282
73283
73284
73285
73286
73287
73288
73289
73290
73291
73292
73293
73294
73295
73296
73297
73298
73299
73300
73301
73302
73303
73304
73305
73306
73307
73308
73309
73310
73311
73312
73313
73314
73315
73316
73317
73318
73319
73320
73321
73322
73323
73324
73325
73326
73327
73328
73329
73330
73331
73332
73333
73334
73335
73336
73337
73338
73339
73340
73341
73342
73343
73344
73345
73346
73347
73348
73349
73350
73351
73352
73353
73354
73355
73356
73357
73358
73359
73360
73361
73362
73363
73364
73365
73366
73367
73368
73369
73370
73371
73372
73373
73374
73375
73376
73377
73378
73379
73380
73381
73382
73383
73384
73385
73386
73387
73388
73389
73390
73391
73392
73393
73394
73395
73396
73397
73398
73399
73400
73401
73402
73403
73404
73405
73406
73407
73408
73409
73410
73411
73412
73413
73414
73415
73416
73417
73418
73419
73420
73421
73422
73423
73424
7342

74637
74638
74639
74640
74641
74642
74643
74644
74645
74646
74647
74648
74649
74650
74651
74652
74653
74654
74655
74656
74657
74658
74659
74660
74661
74662
74663
74664
74665
74666
74667
74668
74669
74670
74671
74672
74673
74674
74675
74676
74677
74678
74679
74680
74681
74682
74683
74684
74685
74686
74687
74688
74689
74690
74691
74692
74693
74694
74695
74696
74697
74698
74699
74700
74701
74702
74703
74704
74705
74706
74707
74708
74709
74710
74711
74712
74713
74714
74715
74716
74717
74718
74719
74720
74721
74722
74723
74724
74725
74726
74727
74728
74729
74730
74731
74732
74733
74734
74735
74736
74737
74738
74739
74740
74741
74742
74743
74744
74745
74746
74747
74748
74749
74750
74751
74752
74753
74754
74755
74756
74757
74758
74759
74760
74761
74762
74763
74764
74765
74766
74767
74768
74769
74770
74771
74772
74773
74774
74775
74776
74777
74778
74779
74780
74781
74782
74783
74784
74785
74786
74787
74788
74789
74790
74791
74792
74793
74794
74795
74796
74797
74798
74799
74800
74801
74802
7480

In [1]:
from pydriller.git_repository import GitRepository
from pydriller import RepositoryMining
repo_updated = "/Users/sanika/Downloads/Master Thesis/repo_updated/pytest"
gr = GitRepository("/Users/sanika/Downloads/Master Thesis/repo_updated/pytest")

buggy_list = [] 
count = 0
for commit in RepositoryMining(repo_updated,
                                   only_modifications_with_file_types=['.py'] ).traverse_commits():
    if "fix" in commit.msg:
        commit1 = gr.get_commit(commit.hash)
        buggy_commits = gr.get_commits_last_modified_lines(commit1)
        for x in buggy_commits:
            buggy_list.append(x)
            count += 1
            print(count,x)
                        

1 7cf982468099472bc6b7cb78cfa7d49336d03449
2 98c1c1809cbbfd53d88e3c03fffd8b972bab730b
3 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
4 eac302140539c9f118d05247e3a9359753d3cb09
5 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
6 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
7 7cf982468099472bc6b7cb78cfa7d49336d03449
8 d929f633a75cf4f6fba9a98fc16d6b205ee8a519
9 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
10 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
11 ed55fea1437a9d7a6f3f9b650d543369f4b7d592
12 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
13 039ec87c64b1236c7f0eef6343c58059cecea5d9
14 7cf982468099472bc6b7cb78cfa7d49336d03449
15 9dc7fbbd37426a8a8d0b7e93f268cf91b213edbd
16 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
17 3d70646122d36e1021f54c85d618e47e1c2589c5
18 b67ef229df78138b9c621b139f4940606c866460
19 ba07a8769f3be6d475a6931a7cd9e6ec0f9ba025
20 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
21 3c08f6a614da514a1c2d74407f181bf33c73e506
22 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
23 b67ef229df78138b9c621b139f4940606c8664

186 abc8cf09aad89e3e90ca0d3f40aa2fd7d5a6e0c4
187 314d1038849281ffdf7a1d4edbaa7cbf52d51003
188 a999dc84721e15c169a26f61674d32d5b30a6ce6
189 abc8cf09aad89e3e90ca0d3f40aa2fd7d5a6e0c4
190 abc8cf09aad89e3e90ca0d3f40aa2fd7d5a6e0c4
191 9515dafb1cefef3b3730685a4d15e4267b71b7fa
192 c17a09adafeb8bc36b70f182c904ce7bcfa0e97e
193 c17a09adafeb8bc36b70f182c904ce7bcfa0e97e
194 abc8cf09aad89e3e90ca0d3f40aa2fd7d5a6e0c4
195 3561d2e9ff8bff069ea6a4c6f6694b7dacbff8c9
196 c17a09adafeb8bc36b70f182c904ce7bcfa0e97e
197 c17a09adafeb8bc36b70f182c904ce7bcfa0e97e
198 1764555778d7125697e9a5edbe1260b3b155f466
199 c17a09adafeb8bc36b70f182c904ce7bcfa0e97e
200 abc8cf09aad89e3e90ca0d3f40aa2fd7d5a6e0c4
201 c17a09adafeb8bc36b70f182c904ce7bcfa0e97e
202 620805c4d2caf565973ef0a6380ccca9403133ed
203 4115e975d1d6bb33d3a6d671b82dabaafd0a50ce
204 abc8cf09aad89e3e90ca0d3f40aa2fd7d5a6e0c4
205 1dc8555ca028d7b155299f411ea3a16bb40d67ab
206 b5a1f95856472ef867effaf97ba735c54caf6ff5
207 a743caef188126af1bb40329c1ec7f4ba28af608
208 c3e5ca

369 fcaefb841ba05c9f6e4c8b38e5ceb9235fb9bc47
370 78d1836d805898340878bd08cd9cf907bf1fd4d3
371 ec34e9423e1b007c2e4119ffa36e5ab8a65f97be
372 763e075bab835962fecea65348727529ec975ead
373 c17a09adafeb8bc36b70f182c904ce7bcfa0e97e
374 65a04bc3be5e452ed0336df30929dc4aa2cde079
375 561a14054cb05a6c876c238e34896594727653c6
376 44abdc9391a058f2709e1946e1892d04f0eec33c
377 ee78661775f7e460380b554a17bcc801b5306abd
378 59bd09ce9ab8c913f1699b86007fb005cc064075
379 afca655202b781a0e2be576e7b2017858f9274ad
380 def623e289aeaff7e27e97284bcb020144cb0cfa
381 236f84d6d4ac299b18b9763c3fbcbc36626114ca
382 4a33940d82ac1cd1d569a62ccecdeb6b9cffd3de
383 c17a09adafeb8bc36b70f182c904ce7bcfa0e97e
384 4aeb929b3c5e5030c903c4b7adfd694af4100baa
385 887a83760032a2b157781e9f78d8850ea4dadf63
386 d16688a1e69015622cb50f8e9de8af634132422e
387 e6234fdb6135099f12ed0bc8a8568d5d3bf77ec4
388 b8926f59daf24635aeac5e3eefcb1080f724b0d8
389 a37c32783dda01d953c40351a49a97770c3f13f0
390 191d02aef23a7a0c5f5086aa9a88e06f84b67b29
391 bcd9ae

563 ee1747fcb4f46e4f53661806f996a280c36ea154
564 6f2cca80aeb9a457e83b079ed06c6d372b545cdc
565 a5e69d2035dc6381b9453e8b8e5761925e1a40de
566 c791610998d4f10bd6b98b04eb165574bda57550
567 43b8bd7df7178fbcef59efe39283d786a8bf4af6
568 1fcd373bd545217adaf8e5cb3e3264f0269e454d
569 24539c379756c5591df382058bbebb840809fbf6
570 3702ca2c710479c81c8db9926f334bdf988d570a
571 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
572 43b8bd7df7178fbcef59efe39283d786a8bf4af6
573 43b8bd7df7178fbcef59efe39283d786a8bf4af6
574 3702ca2c710479c81c8db9926f334bdf988d570a
575 5d2504df0a29a5d15dd7f9469668f007d642ad6e
576 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
577 f636ed8ced651d88097b4fdef0acd0d6cd11b3ac
578 e30aeed876fb936e379a8b10eef2cbc8fca8f08f
579 43b8bd7df7178fbcef59efe39283d786a8bf4af6
580 beb3a127008a76d9e08d039345bf38dde0adb845
581 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
582 f636ed8ced651d88097b4fdef0acd0d6cd11b3ac
583 191d02aef23a7a0c5f5086aa9a88e06f84b67b29
584 22b4ff7f06b7bcd4fcc8f4e82d2cc114e783b7ab
585 845357

759 4ab0f25b050cc6d29e2f68704a88e06ba61ea5aa
760 6d84da39e4b7c44cdb72067287c47148da6ff5c1
761 4035fa632693b6e699679bdeac39255256c1d798
762 43b8bd7df7178fbcef59efe39283d786a8bf4af6
763 47bad98c0756f505c5254056b3d07eb2ec5249b7
764 a4c14e83f068a222ce422e7cfc449530e15736f4
765 33b283327d57ee5bef5e7a55dd6f774d3bf5389b
766 5f3fac94b6234b1d58d1b32562e7f71716817a85
767 c791610998d4f10bd6b98b04eb165574bda57550
768 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
769 36189a7aa7bcd61e1caebf557b3087f17c2257e5
770 5992a8ef21424d7571305a8d7e2a3431ee7e1e23
771 739edc26b4e9894abc932cd225216e0858d9894f
772 60a7556ca3b00527bc2a7666df334146d64b612f
773 4035fa632693b6e699679bdeac39255256c1d798
774 a59d602bcef5572ccb74c3bd9c85d897e05f674d
775 7ab98c1b25d4a13331c81bed11dab54133e88ce7
776 8af3ede092d71e2fc61f13e1af7329717aeefa6a
777 c8119d89b62c07bcd3abf013e42631c34aee3923
778 16aa1571c0873afb32579bfe783aa91b64cec517
779 236f84d6d4ac299b18b9763c3fbcbc36626114ca
780 066f8e854d619a723c491ee427c24f938a6ba0f2
781 ab9f6a

...

9599 57bc14caa0a9715cca597fb7912b2b6f5dd118a5
9600 6e170a4a1c4903b85a47f90e441edc7ff5222589
9601 fadd1a231375c97a9093d50f75aee6890e1ec7fb
9602 703e4b11ba76171eccd3f13e723c47b810ded7ef
9603 219b758949a473507d46f0105b1853319c971b6c
9604 c9bbdf4f108f22df04def1d63a3442b9f3fcda7d
9605 18a47bfd22bb93ad5875281ff82fd0efaf68156e
9606 8f9081248105eaa704fa79abb5e544571ab8c9e8
9607 43544a431ca958b411d6b84c7093f03e48461349
9608 be949f40375662a492a670627233e135bff2906e
9609 749288dcb62bb242ba454ec956ad2ce2ad93b54b
9610 703e4b11ba76171eccd3f13e723c47b810ded7ef
9611 9c5495832cc89c6dd9b89719eb149ee379d6992b
9612 a8afba054a687b2735e9016a3a7335ebdf0846ea
9613 7b1cb885c747dcbbbba3f320e5465bade8a98fa2
9614 7b1cb885c747dcbbbba3f320e5465bade8a98fa2
9615 475119988ce4960bc7356a82d8ea2518d2c6089d
9616 5603a0cd4b6adf4fbf5e490216ebfcc0befbf10a
9617 bd1d17e8de30f6d52f9c4b61df4c6403f8f4f61c
9618 5c8e5acf9d6d1ac03ed8a483acf329a3276c0622
9619 5603a0cd4b6adf4fbf5e490216ebfcc0befbf10a
9620 c89827b9f2b42352a388fa1a013f6

9800 0d00be4f4f54aaef99ef94ca40dfb7b21ca16a0b
9801 1caf6d59070ae24f7d442527209155e62af97b9c
9802 fc61bdd9073d86819eb54c8603384a10c626a9d4
9803 06a49338b2b1be8daed89f779439459a2cb4cbc7
9804 703e4b11ba76171eccd3f13e723c47b810ded7ef
9805 60ff2e85290072554256bf977e4cd5c0553908b8
9806 095ce2ca7fd3828f451a10c191f41517c9dfe34b
9807 5ae59279f48b500625b8cffb8ca3e6f8386aa291
9808 ec2d8223cff2fe83119d142c3b4b365786b3182d
9809 15b9e8ed7dbb62daeace2e4a3fc2d3115a9ea896
9810 0d00be4f4f54aaef99ef94ca40dfb7b21ca16a0b
9811 1caf6d59070ae24f7d442527209155e62af97b9c
9812 fc61bdd9073d86819eb54c8603384a10c626a9d4
9813 06a49338b2b1be8daed89f779439459a2cb4cbc7
9814 703e4b11ba76171eccd3f13e723c47b810ded7ef
9815 60ff2e85290072554256bf977e4cd5c0553908b8
9816 095ce2ca7fd3828f451a10c191f41517c9dfe34b
9817 5ae59279f48b500625b8cffb8ca3e6f8386aa291
9818 ec2d8223cff2fe83119d142c3b4b365786b3182d
9819 15b9e8ed7dbb62daeace2e4a3fc2d3115a9ea896
9820 1cfea5f1b33272a332a6b87c9ae99892be00e2c0
9821 1cfea5f1b33272a332a6b87c9ae99

9988 8c49561470708761f7321504f5e8343811be87ac
9989 703e4b11ba76171eccd3f13e723c47b810ded7ef
9990 10bf6aac76d5060a0db4a94871d6dcf0a1cce397
9991 505c3340bf852f7636eae1490a8638f5ffab266d
9992 a98270eac078db7d78a50b35f0fcf9bdb8bf888f
9993 1aac64573fab858e9c216a8faf9e802fbb1b8303
9994 29bb0eda276fc3808264a1b070544339c1eee393
9995 10bf6aac76d5060a0db4a94871d6dcf0a1cce397
9996 bf0b70aa6694e4cae178d41320320d170d12193e
9997 c8e7d1ae349343f17b62d84aa0b3aef50393af7e
9998 65f5383106d4fbe7680bf154673198d68f590f9a
9999 db5649ec6a8fcd9ef148b9f77797a22cb9f3cda1
10000 d42f1e87c3c4b55ef5faa1a11a4f3083860c864d
10001 9769bc05c626dd0401e691c2c9c68f774fb990c3
10002 360d608da4e11c66539967e625c70e6cae31395e
10003 d42f1e87c3c4b55ef5faa1a11a4f3083860c864d
10004 9769bc05c626dd0401e691c2c9c68f774fb990c3
10005 360d608da4e11c66539967e625c70e6cae31395e
10006 1dd5f088faed8248f06b3ef59b8e38b0c249e4c3
10007 cc90bcce4c3d6bc55aeb85a6450020cb4a2e49f2
10008 8c49561470708761f7321504f5e8343811be87ac
10009 8c49561470708761f73

10190 a0b0c37febd8752cdf1cb74fe80cd1e180056ba8
10191 a0b0c37febd8752cdf1cb74fe80cd1e180056ba8
10192 8651d880a06c078d8c04ec8c99df1a05bc9826d7
10193 ab39502c98f40c03a2956dcf6c6b9daf5ee41a63
10194 ab86dea5294a25531e1cf207bed09fc4e8c2284b
10195 4b709037abf86e59f4227a8aad4ba8e1c64a0634
10196 617e510b6e4fb2e95a1486c8f30dd1e7583be248
10197 703e4b11ba76171eccd3f13e723c47b810ded7ef
10198 3f5e9ea71e997746f89c71507888414f9477e2b9
10199 a2891420de5a020fbcc09ee0af3124325334fc6d
10200 2a6a1ca07dac49cc7ceb64543ea16ad457f1a1dd
10201 d67d68f6d36ff7feec745c92f7dc6d52e5cce013
10202 d73e689991ec3867aee56f2efe5212a6a6e18924
10203 d17ea7a9c0f74da2a48e5c59ccb3af7a6aaa0e9e
10204 616d8251f34bdd9a4fe7e0bac265412ab8cea961
10205 8c49561470708761f7321504f5e8343811be87ac
10206 741f0fedd1d8661ed671fdc7a1091dd42203a56f
10207 cc6eb9f83c9a12cc4416ef03c022c3eb4bc2ee22
10208 99180939febb6ca4b3ff788cf336b065046653f2
10209 a912d3745bcbbc49999dc9becf0d5545849d44ea
10210 090e26051738314317fbe7e6fc603e1cf39d6c87
10211 82ba645

10367 8c49561470708761f7321504f5e8343811be87ac
10368 ef112fd7dd8ee220865f7569c0ecc4fa5eb24464
10369 5436e4299075236023e8875c563a7c74cad9bb2a
10370 8c49561470708761f7321504f5e8343811be87ac
10371 65bd1b8a9316ee4ab514941eb883f89d20caa8b8
10372 65bd1b8a9316ee4ab514941eb883f89d20caa8b8
10373 3da28067f306582a10b798ba527356d62c1f4f86
10374 2d06ae0f65f88ca21d3f2ad8fe51286548dafab5
10375 b0d45267c58859bcb79f7ab980f4b410c4bbd109
10376 a912d3745bcbbc49999dc9becf0d5545849d44ea
10377 62db3f7abcec8cfdcd8526310f866d0fd08d8025


In [2]:
len(buggy_list)

10377

In [3]:
import pandas as pd
df = pd.read_csv("final_dataset_03.csv")
df.loc[df['CommitID'].isin(buggy_list)]

,CommitID,filename,complexity,token_count,loc,lloc,sloc,comments,multi,blank,...,effort,time,bugs,change_type,added,removed,introspection,object_changes,code_generation,library_loading
0,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0
1,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0
2,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,2,2,2,0,0,0,...,0.000000,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0
3,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0
4,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75771,b0d45267c58859bcb79f7ab980f4b410c4bbd109,/Users/sanika/Downloads/Master Thesis/repo_upd...,69.0,1502.0,402,202,227,24,74,80,...,3969.692308,220.538462,0.188000,ModificationType.MODIFY,1,1,95550,37574,7721,50734
75772,b0d45267c58859bcb79f7ab980f4b410c4bbd109,/Users/sanika/Downloads/Master Thesis/repo_upd...,65.0,2178.0,53,23,39,0,0,13,...,39.863137,2.214619,0.013288,ModificationType.MODIFY,1,1,95550,37574,7721,50735
75773,b0d45267c58859bcb79f7ab980f4b410c4bbd109,/Users/sanika/Downloads/Master Thesis/repo_upd...,65.0,2178.0,515,298,307,33,80,99,...,5527.038451,307.057692,0.251229,ModificationType.MODIFY,1,1,95560,37574,7721,50736
75774,b0d45267c58859bcb79f7ab980f4b410c4bbd109,/Users/sanika/Downloads/Master Thesis/repo_upd...,294.0,6567.0,72,33,56,0,4,12,...,1.000000,0.055556,0.000667,ModificationType.MODIFY,1,1,95560,37574,7721,50737


In [4]:
df

,CommitID,filename,complexity,token_count,loc,lloc,sloc,comments,multi,blank,...,effort,time,bugs,change_type,added,removed,introspection,object_changes,code_generation,library_loading
0,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0
1,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0
2,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,2,2,2,0,0,0,...,0.000000,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0
3,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0
4,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75870,ef437ea44831c949650376c24f925a023f4192db,/Users/sanika/Downloads/Master Thesis/repo_upd...,119.0,3209.0,1523,757,1297,19,12,193,...,19053.864458,1058.548025,0.888260,ModificationType.MODIFY,1,1,95700,37606,7739,50797
75871,ef437ea44831c949650376c24f925a023f4192db,/Users/sanika/Downloads/Master Thesis/repo_upd...,119.0,3209.0,701,378,504,33,58,109,...,10027.894179,557.105232,0.448949,ModificationType.MODIFY,1,1,95700,37607,7740,50798
75872,a8fc056aad47070bbf60c266409f3eafd71c434c,/Users/sanika/Downloads/Master Thesis/repo_upd...,124.0,3511.0,77,34,69,0,0,8,...,387.820512,21.545584,0.071323,ModificationType.MODIFY,20,10,95700,37607,7740,50799
75873,a8fc056aad47070bbf60c266409f3eafd71c434c,/Users/sanika/Downloads/Master Thesis/repo_upd...,124.0,3511.0,712,438,573,51,9,84,...,11988.288660,666.016037,0.454913,ModificationType.MODIFY,20,10,95710,37608,7740,50800


In [5]:
# Creating new dataframe 

df1 = pd.DataFrame(df, columns = ['CommitID','filename','complexity','token_count','loc','lloc','sloc','comments',
          'multi','blank','code_comment','h1','h2','N1','N2','vocabulary','length',
          'calculated_length','volume', 'difficulty','effort','time','bugs',
          'change_type','added','removed','introspection','object_changes',
          'code_generation','library_loading']) 

# Generate result using pandas 
result = [] 
for value in df1["CommitID"]:
    if value in buggy_list:
        result.append(True)
    else:
        result.append(False)
    
df1["defect"] = result 
df1 

,CommitID,filename,complexity,token_count,loc,lloc,sloc,comments,multi,blank,...,time,bugs,change_type,added,removed,introspection,object_changes,code_generation,library_loading,defect
0,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
1,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
2,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,2,2,2,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
3,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
4,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75870,ef437ea44831c949650376c24f925a023f4192db,/Users/sanika/Downloads/Master Thesis/repo_upd...,119.0,3209.0,1523,757,1297,19,12,193,...,1058.548025,0.888260,ModificationType.MODIFY,1,1,95700,37606,7739,50797,False
75871,ef437ea44831c949650376c24f925a023f4192db,/Users/sanika/Downloads/Master Thesis/repo_upd...,119.0,3209.0,701,378,504,33,58,109,...,557.105232,0.448949,ModificationType.MODIFY,1,1,95700,37607,7740,50798,False
75872,a8fc056aad47070bbf60c266409f3eafd71c434c,/Users/sanika/Downloads/Master Thesis/repo_upd...,124.0,3511.0,77,34,69,0,0,8,...,21.545584,0.071323,ModificationType.MODIFY,20,10,95700,37607,7740,50799,False
75873,a8fc056aad47070bbf60c266409f3eafd71c434c,/Users/sanika/Downloads/Master Thesis/repo_upd...,124.0,3511.0,712,438,573,51,9,84,...,666.016037,0.454913,ModificationType.MODIFY,20,10,95710,37608,7740,50800,False


In [6]:
#writing release filter labelled data to CSV
df1.to_csv('final_dataset_03.csv')

In [7]:
df1

,CommitID,filename,complexity,token_count,loc,lloc,sloc,comments,multi,blank,...,time,bugs,change_type,added,removed,introspection,object_changes,code_generation,library_loading,defect
0,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
1,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
2,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,2,2,2,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
3,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
4,5992a8ef21424d7571305a8d7e2a3431ee7e1e23,/Users/sanika/Downloads/Master Thesis/repo_upd...,0.0,685.0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75870,ef437ea44831c949650376c24f925a023f4192db,/Users/sanika/Downloads/Master Thesis/repo_upd...,119.0,3209.0,1523,757,1297,19,12,193,...,1058.548025,0.888260,ModificationType.MODIFY,1,1,95700,37606,7739,50797,False
75871,ef437ea44831c949650376c24f925a023f4192db,/Users/sanika/Downloads/Master Thesis/repo_upd...,119.0,3209.0,701,378,504,33,58,109,...,557.105232,0.448949,ModificationType.MODIFY,1,1,95700,37607,7740,50798,False
75872,a8fc056aad47070bbf60c266409f3eafd71c434c,/Users/sanika/Downloads/Master Thesis/repo_upd...,124.0,3511.0,77,34,69,0,0,8,...,21.545584,0.071323,ModificationType.MODIFY,20,10,95700,37607,7740,50799,False
75873,a8fc056aad47070bbf60c266409f3eafd71c434c,/Users/sanika/Downloads/Master Thesis/repo_upd...,124.0,3511.0,712,438,573,51,9,84,...,666.016037,0.454913,ModificationType.MODIFY,20,10,95710,37608,7740,50800,False


In [3]:
# importing pandas
import pandas as pd

#fetch data and add headers
filepath = 'final_dataset_03.csv'
column = ['complexity','token_count','loc','lloc','sloc','comments',
          'multi','blank','code_comment','h1','h2','N1','N2','vocabulary','length',
          'calculated_length','volume', 'difficulty','effort','time','bugs',
          'change_type','added','removed','introspection','object_changes',
          'code_generation','library_loading','defect']

df = pd.read_csv(filepath, index_col=False, usecols = column)
df

,complexity,token_count,loc,lloc,sloc,comments,multi,blank,code_comment,h1,...,time,bugs,change_type,added,removed,introspection,object_changes,code_generation,library_loading,defect
0,0.0,685.0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
1,0.0,685.0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
2,0.0,685.0,2,2,2,0,0,0,2,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
3,0.0,685.0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
4,0.0,685.0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75870,119.0,3209.0,1523,757,1297,19,12,193,1542,12,...,1058.548025,0.888260,ModificationType.MODIFY,1,1,95700,37606,7739,50797,False
75871,119.0,3209.0,701,378,504,33,58,109,734,13,...,557.105232,0.448949,ModificationType.MODIFY,1,1,95700,37607,7740,50798,False
75872,124.0,3511.0,77,34,69,0,0,8,77,3,...,21.545584,0.071323,ModificationType.MODIFY,20,10,95700,37607,7740,50799,False
75873,124.0,3511.0,712,438,573,51,9,84,763,14,...,666.016037,0.454913,ModificationType.MODIFY,20,10,95710,37608,7740,50800,False


In [4]:
# dropiing the duplicate rows but keeping the first row that has a duplicate below in the dataset
df.drop_duplicates(keep='first', inplace=True)
df

,complexity,token_count,loc,lloc,sloc,comments,multi,blank,code_comment,h1,...,time,bugs,change_type,added,removed,introspection,object_changes,code_generation,library_loading,defect
0,0.0,685.0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
2,0.0,685.0,2,2,2,0,0,0,2,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,0,0,0,True
16,0.0,685.0,99,48,90,1,3,5,100,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,1,0,1,True
17,0.0,685.0,8,5,5,2,0,1,10,0,...,0.000000,0.000000,ModificationType.ADD,129,0,0,1,0,2,True
18,0.0,685.0,1217,716,841,84,126,174,1301,14,...,1149.312005,0.847665,ModificationType.ADD,129,0,10,3,0,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75870,119.0,3209.0,1523,757,1297,19,12,193,1542,12,...,1058.548025,0.888260,ModificationType.MODIFY,1,1,95700,37606,7739,50797,False
75871,119.0,3209.0,701,378,504,33,58,109,734,13,...,557.105232,0.448949,ModificationType.MODIFY,1,1,95700,37607,7740,50798,False
75872,124.0,3511.0,77,34,69,0,0,8,77,3,...,21.545584,0.071323,ModificationType.MODIFY,20,10,95700,37607,7740,50799,False
75873,124.0,3511.0,712,438,573,51,9,84,763,14,...,666.016037,0.454913,ModificationType.MODIFY,20,10,95710,37608,7740,50800,False


In [5]:
df.dtypes

complexity           float64
token_count          float64
loc                    int64
lloc                   int64
sloc                   int64
comments               int64
multi                  int64
blank                  int64
code_comment           int64
h1                     int64
h2                     int64
N1                     int64
N2                     int64
vocabulary             int64
length                 int64
calculated_length    float64
volume               float64
difficulty           float64
effort               float64
time                 float64
bugs                 float64
change_type           object
added                  int64
removed                int64
introspection          int64
object_changes         int64
code_generation        int64
library_loading        int64
defect                  bool
dtype: object

In [6]:
#Added, Deleted, Modified, or Renamed
categories =['ModificationType.ADD', 'ModificationType.DELETE', 
            'ModificationType.MODIFY','ModificationType.RENAME']
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df.change_type = encoder.fit_transform(df.change_type.values.reshape(-1, 1))
df

,complexity,token_count,loc,lloc,sloc,comments,multi,blank,code_comment,h1,...,time,bugs,change_type,added,removed,introspection,object_changes,code_generation,library_loading,defect
0,0.0,685.0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.0,129,0,0,0,0,0,True
2,0.0,685.0,2,2,2,0,0,0,2,0,...,0.000000,0.000000,0.0,129,0,0,0,0,0,True
16,0.0,685.0,99,48,90,1,3,5,100,0,...,0.000000,0.000000,0.0,129,0,0,1,0,1,True
17,0.0,685.0,8,5,5,2,0,1,10,0,...,0.000000,0.000000,0.0,129,0,0,1,0,2,True
18,0.0,685.0,1217,716,841,84,126,174,1301,14,...,1149.312005,0.847665,0.0,129,0,10,3,0,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75870,119.0,3209.0,1523,757,1297,19,12,193,1542,12,...,1058.548025,0.888260,2.0,1,1,95700,37606,7739,50797,False
75871,119.0,3209.0,701,378,504,33,58,109,734,13,...,557.105232,0.448949,2.0,1,1,95700,37607,7740,50798,False
75872,124.0,3511.0,77,34,69,0,0,8,77,3,...,21.545584,0.071323,2.0,20,10,95700,37607,7740,50799,False
75873,124.0,3511.0,712,438,573,51,9,84,763,14,...,666.016037,0.454913,2.0,20,10,95710,37608,7740,50800,False


In [7]:
#remove rows where LOC = 0, and complexity = 0
col = ['complexity','loc']
df = df.replace(0, pd.np.nan).dropna(axis=0,how='any',subset=col).fillna(0).astype(int)
df

,complexity,token_count,loc,lloc,sloc,comments,multi,blank,code_comment,h1,...,time,bugs,change_type,added,removed,introspection,object_changes,code_generation,library_loading,defect
381,19,473,182,127,129,0,0,53,182,5,...,51,0,0,80,0,170,62,16,132,1
382,19,473,1165,710,800,67,119,181,1232,18,...,2026,1,0,80,0,180,62,17,133,1
383,12,234,1392,870,1177,35,17,170,1427,14,...,3047,2,0,40,0,180,65,18,134,1
384,62,1372,773,419,590,28,0,166,801,10,...,754,0,0,282,0,180,65,19,135,1
385,62,1372,421,273,302,44,34,57,465,17,...,684,0,0,282,0,180,65,20,136,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75870,119,3209,1523,757,1297,19,12,193,1542,12,...,1058,0,2,1,1,95700,37606,7739,50797,0
75871,119,3209,701,378,504,33,58,109,734,13,...,557,0,2,1,1,95700,37607,7740,50798,0
75872,124,3511,77,34,69,0,0,8,77,3,...,21,0,2,20,10,95700,37607,7740,50799,0
75873,124,3511,712,438,573,51,9,84,763,14,...,666,0,2,20,10,95710,37608,7740,50800,0


In [9]:
#writing release filter labelled data to CSV
df.to_csv('dataset_processed_03.csv',index=False)

In [10]:
df

,complexity,token_count,loc,lloc,sloc,comments,multi,blank,code_comment,h1,...,time,bugs,change_type,added,removed,introspection,object_changes,code_generation,library_loading,defect
381,19,473,182,127,129,0,0,53,182,5,...,51,0,0,80,0,170,62,16,132,1
382,19,473,1165,710,800,67,119,181,1232,18,...,2026,1,0,80,0,180,62,17,133,1
383,12,234,1392,870,1177,35,17,170,1427,14,...,3047,2,0,40,0,180,65,18,134,1
384,62,1372,773,419,590,28,0,166,801,10,...,754,0,0,282,0,180,65,19,135,1
385,62,1372,421,273,302,44,34,57,465,17,...,684,0,0,282,0,180,65,20,136,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75870,119,3209,1523,757,1297,19,12,193,1542,12,...,1058,0,2,1,1,95700,37606,7739,50797,0
75871,119,3209,701,378,504,33,58,109,734,13,...,557,0,2,1,1,95700,37607,7740,50798,0
75872,124,3511,77,34,69,0,0,8,77,3,...,21,0,2,20,10,95700,37607,7740,50799,0
75873,124,3511,712,438,573,51,9,84,763,14,...,666,0,2,20,10,95710,37608,7740,50800,0
